# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
import time
from sklearn.cross_validation import train_test_split
from datetime import datetime

tf.logging.set_verbosity(tf.logging.INFO)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data loading

In [2]:
width = 640
height = 480
channel = 3
total_pix = width * height * channel
num_class = 2

In [3]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out

In [4]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]))

1780


In [5]:
tr_data_dir = data_process(data_dir1, 0.2)

In [6]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    tr_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    tr_label[i] = tr_data_dir['train'][i][1]
    
    

    
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    ev_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

# CNN model

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x, W, n):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=n)

def max_pool_2x2(x, n):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME', name = n)

In [9]:
def cnn_model(X, keep_prob, reuse):
    width = 640
    height = 480
    channel = 3
    total_pix = width * height * channel
    num_class = 2
    
    with tf.variable_scope('CNN', reuse=reuse):
    
        x_img = tf.reshape(X, [-1, height, width, channel])

        wc1 = weight_variable([5, 5, 3, 48])
        bc1 = bias_variable([48])
        hc1 = tf.nn.relu(conv2d(x_img, wc1, 'conv1') + bc1)
        hp1 = max_pool_2x2(hc1, 'max1')

        wc2 = weight_variable([5, 5, 48, 96])
        bc2 = bias_variable([96])
        hc2 = tf.nn.relu(conv2d(hp1, wc2, 'conv2') + bc2)
        hp2 = max_pool_2x2(hc2, 'max2')

        wc3 = weight_variable([5, 5, 96, 96])
        bc3 = bias_variable([96])
        hc3 = tf.nn.relu(conv2d(hp2, wc3, 'conv3') + bc3)
        hp3 = max_pool_2x2(hc3, 'max3')

        wc4 = weight_variable([5, 5, 96, 96])
        bc4 = bias_variable([96])
        hc4 = tf.nn.relu(conv2d(hp3, wc4, 'conv4') + bc4)
        hp4 = max_pool_2x2(hc4, 'max4')

        wc5 = weight_variable([5, 5, 96, 96])
        bc5 = bias_variable([96])
        hc5 = tf.nn.relu(conv2d(hp4, wc5, 'conv5') + bc5)
        hp5 = max_pool_2x2(hc5, 'max5')

        wf1 = weight_variable([15*20*96, 1024])
        bf1 = bias_variable([1024])
        hf_flat = tf.reshape(hp5, [-1, 15*20*96])
        hf1 = tf.nn.relu(tf.matmul(hf_flat, wf1) + bf1)
        hf_drop = tf.nn.dropout(hf1, keep_prob)

        wf2 = weight_variable([1024, num_class])
        bf2 = bias_variable([num_class])

        y_conv = tf.matmul(hf_drop, wf2) + bf2
    
    return y_conv

# Multi-GPU part

In [10]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [11]:
# By default, all variables will be placed on '/gpu:0'
# So we need a custom device function, to assign all variables to '/cpu:0'
# Note: If GPUs are peered, '/gpu:0' can be a faster option
PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']

def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device

    return _assign

# Training

In [12]:
today = datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(today)

2018-05-03-08-58-44


In [13]:
learning_r = 0.001
batch_s = 16
epochs = 1
num_gpus = 3
total_steps = int((tr_data.shape[0]/batch_s*epochs)/num_gpus)
print(total_steps)
disp_step = 10

54


In [14]:
# Place all ops on CPU by default
with tf.device('/cpu:0'):
    tower_grads = []
    reuse_vars = False

    # tf Graph input
    X = tf.placeholder(tf.float32, shape = [None, total_pix])
    Y = tf.placeholder(tf.int32, shape = [None, 1])
    keep_prob = tf.placeholder(tf.float32)
    
    # Loop over all GPUs and construct their own computation graph
    for i in range(num_gpus):
        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):
            
            # Split data between GPUs
            _x = X[i * batch_s: (i+1) * batch_s]
            _y = Y[i * batch_s: (i+1) * batch_s]
            
            # Because Dropout have different behavior at training and prediction time, we
            # need to create 2 distinct computation graphs that share the same weights.

            # Create a graph for training
            logits_train = cnn_model(_x, keep_prob=0.5, reuse=reuse_vars)
            # Create another graph for testing that reuse the same weights
            logits_test = cnn_model(_x, keep_prob=1.0, reuse=True)
            
            # Define loss and optimizer (with train logits, for dropout to take effect)
            loss = tf.losses.sparse_softmax_cross_entropy(labels = Y, logits = logits_train)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_r)
            grads = optimizer.compute_gradients(loss)
            
            # Only first GPU compute accuracy
            if i == 0:
                # Evaluate model (with test logits, for dropout to be disabled)
                correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
                
            reuse_vars = True
            tower_grads.append(grads)
            
    tower_grads = average_gradients(tower_grads)
    train_op = optimizer.apply_gradients(tower_grads)
    
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    # Start Training
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)

        # Keep training until reach max iterations
        for step in range(1, total_steps + 1):
            # Get a batch for each GPU
            tr_batch = {}

            if step < total_steps+1:
                tr_batch['data'] = tr_data[(i-1)*num_gpus*batch_s:i*num_gpus*batch_s][:]
                tr_batch['label'] = tr_label[(i-1)*num_gpus*batch_s:i*num_gpus*batch_s]

            else:
                tr_batch['data'] = tr_data[(i-1)*num_gpus*batch_s:][:]
                tr_batch['label'] = tr_label[(i-1)*num_gpus*batch_s:]

            ts = time.time()
            sess.run(train_op, feed_dict={X: tr_batch['data'],
                                          Y: tr_batch['label']})
            te = time.time() - ts
            
            if step % disp_step == 0 or step == 1:
                loss, acc = sess.run([loss, accuracy], feed_dict={X: tr_batch['data'],
                                                                  Y: tr_batch['label']})
                
                print("Step " + str(step) + ": Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc) + ", %i Examples/sec" % int(len(batch_x)/te))
            # Calculate accuracy for MNIST test images
        
        
        ev_accuracy = np.zeros((len(ev_label), 1))
        for k in range(len(ev_label)):
            ev_batch = {}
            ev_batch['data'] = ev_data[k:(k+1)*batch_s][:]
            ev_batch['label'] = ev_label[k:(k+1)*batch_s]
            ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_batch['data'],
                                                           Y: ev_batch['label']})
            
        print(np.mean(ev_accuracy))

ValueError: Tried to convert 'input' to a tensor and failed. Error: None values not supported.

In [ ]:
#for d in ['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3']:
for d in ['/gpu:3']:
    with tf.device(d):
        loss = tf.losses.sparse_softmax_cross_entropy(labels = Y, logits = y_conv)
        train_step = tf.train.GradientDescentOptimizer(learning_rate = learning_r).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
loss = tf.losses.sparse_softmax_cross_entropy(labels = Y, logits = y_conv)
train_step = tf.train.GradientDescentOptimizer(learning_rate = learning_r).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
batch_s = 16
epochs = 1
total_steps = int(tr_data.shape[0]/batch_s*epochs)
print(total_steps)

In [ ]:
ev_accuracy = np.zeros((len(ev_label), 1))

In [ ]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(total_steps):
        
        tr_batch = {}

        if i < total_steps-1:
            tr_batch['data'] = tr_data[i*batch_s:(i+1)*batch_s][:]
            tr_batch['label'] = tr_label[i*batch_s:(i+1)*batch_s]

        else:
            tr_batch['data'] = tr_data[i*batch_s:][:]
            tr_batch['label'] = tr_label[i*batch_s:]
            
        train_step.run(feed_dict={X: tr_batch['data'],
                                  Y: tr_batch['label'],
                                  keep_prob: 0.5})

        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict = {X: tr_batch['data'],
                                                        Y: tr_batch['label'],
                                                        keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        if i == total_steps-1:
            print("Training is finished. Start evaluation.")
            
            
    for j in range(len(ev_label)):
        ev_accuracy[j] = accuracy.eval(feed_dict={X: ev_data[j:j+1],
                                                  Y: ev_label[j:j+1],
                                                  keep_prob: 1.0})

In [ ]:
print(np.mean(ev_accuracy))

In [ ]:
    logits = tf.layers.dense(inputs=dropout, units=2)
    #print(logits.shape)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
total_pix = 640*480*3
x = tf.placeholder(tf.float32, shape=[None, total_pix])
input_layer = tf.reshape(x, [-1, 480, 640, 3])

# Convolutional Layer #1
# Computes 48 features using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 480, 640, 3]
# Output Tensor Shape: [batch_size, 480, 640, 48]
conv1 = tf.layers.conv2d(

    inputs=input_layer,
    filters=48,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)
print(conv1.shape)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
print(pool1.shape)

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 480, 640, 3])

    # Convolutional Layer #1
    # Computes 48 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 480, 640, 3]
    # Output Tensor Shape: [batch_size, 480, 640, 48]
    conv1 = tf.layers.conv2d(
        
        inputs=input_layer,
        filters=48,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    #print(conv1.shape)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 480, 640, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 48]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    #print(pool1.shape)

    # Convolutional Layer #2
    # Computes 96 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 240, 320, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 96]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    #print(conv2.shape)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 240, 320, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    #print(pool2.shape)

    
    
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    #print(conv3.shape)
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    #print(pool3.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    #print(conv4.shape)
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    #print(pool4.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    #print(conv5.shape)
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 15, 20, 96]
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)
    #print(pool5.shape)
    
    
    
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 15, 20, 96]
    # Output Tensor Shape: [batch_size, 15 * 20 * 96]
    pool5_flat = tf.reshape(pool5, [-1, 15 * 20 * 96])
    #print(pool5_flat.shape)


    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 15 * 20 * 96]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool5_flat, units=1024, activation=tf.nn.relu)
    #print(dense.shape)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 1]
    logits = tf.layers.dense(inputs=dropout, units=2)
    #print(logits.shape)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out
        

## Code Run


### Training
Batch size: 128  
Epoch: 5  
Shuffle: True  
Step: 20000

## Loading dataset
Load dataset and split them into training data and evaluation data

In [ ]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]))

In [ ]:
image_dir2 = '/data_oh/train-images/train-images_128/'
class_name2 = ['bite', 'cut', 'normal']

data_dir2 = {}

for i in range(len(class_name2)):
    temp_dir = image_dir2 + class_name2[i]
    
    if i == 2:
        label=0
    else:
        label=1
    img_files = [[os.path.join(temp_dir, f), np.int16(label)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir2[i] = img_files
print(len(data_dir2[0]))

In [ ]:
image_dir3 = '/data_oh/test-images/test-images_128/'
class_name3 = ['bite', 'cut', 'normal']

data_dir3 = {}

for i in range(len(class_name3)):
    temp_dir = image_dir3 + class_name3[i]
    
    if i == 2:
        label=0
    else:
        label=1
    img_files = [[os.path.join(temp_dir, f), np.int16(label)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir3[i] = img_files
print(len(data_dir3[0]))

In [ ]:
tr_data_dir = data_process(data_dir1, 0.2)

In [ ]:
tr_temp = tr_data_dir['train']
tr_temp.extend(data_dir2[2])
tr_temp.extend(data_dir2[1])
tr_temp.extend(data_dir2[0])
random.shuffle(tr_temp)
print(tr_temp)

In [ ]:
print(len(tr_temp))
tr_data_dir['train'] = tr_temp
print(len(tr_data_dir['train']), len(tr_data_dir['test']))

In [ ]:
te_data_dir = data_dir3[0]
te_data_dir.extend(data_dir3[1])
te_data_dir.extend(data_dir3[2])
print(te_data_dir)

In [ ]:
im_width = 640
im_height = 480
num_chan = 3
total_pix = im_width * im_height * num_chan

In [ ]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

In [ ]:
for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    tr_data[i][:] = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC).flatten()
    tr_label[i] = tr_data_dir['train'][i][1]

In [ ]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    ev_data[i][:] = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC).flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

Batch 16  
Epoch 45  
Training steps 7400

In [ ]:
today = datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(today)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = tf.Session(config=config)

# Create the Estimator
net_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/models/CNN2600_blend_%s" % today)

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

batch_s = 16
epochs = 15
tr_steps = int(tr_data.shape[0]/batch_s*epochs)
in_steps = 1
print(tr_steps)

for d in ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2']:
    with tf.device(d):

        # Train the model
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": tr_data},
            y=tr_label,
            batch_size=batch_s,
            num_epochs=epochs,
            shuffle=True)
        net_classifier.train(
            input_fn=train_input_fn,
            steps=tr_steps,
            hooks=[logging_hook])

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.Session(config=config)

ev_results = np.zeros((len(ev_label), 1))
with tf.device('/device:GPU:3'):

    # Evaluate the model and print results
    for i in range(len(ev_label)):
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": ev_data[i:i+1]},
            y=ev_label[i:i+1],
            num_epochs=1,
            shuffle=False)
        eval_results = net_classifier.evaluate(input_fn=eval_input_fn)
        
        ev_results[i] = eval_results["accuracy"]

In [ ]:
print(np.mean(ev_results))

In [ ]:
fail_idx = np.where(ev_results == 0)
print(fail_idx[0])
print(len(fail_idx[0]))

In [ ]:
for i in range(len(fail_idx[0])):

    temp_img = cv2.imread(tr_data_dir['test'][fail_idx[0][i]][0])
    
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.axis("off")
    plt.imshow(cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
print(te_data_dir[3][0])

In [ ]:
te_data = np.zeros((len(te_data_dir), total_pix), dtype=np.float32)
te_label = np.zeros((len(te_data_dir), 1), dtype=np.int32)

for i in range(len(te_data_dir)):
    img = cv2.imread(te_data_dir[i][0])
    te_data[i][:] = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC).flatten()
    te_label[i] = te_data_dir[i][1]

In [ ]:
te_results = np.zeros((len(te_label), 1))
with tf.device('/device:GPU:3'):

    # Evaluate the model and print results
    for i in range(len(te_label)):
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": te_data[i:i+1]},
            y=te_label[i:i+1],
            num_epochs=1,
            shuffle=False)
        eval_results = net_classifier.evaluate(input_fn=eval_input_fn)
        
        te_results[i] = eval_results["accuracy"]

In [ ]:
print(np.mean(te_results))

In [ ]:
fail_idx2 = np.where(te_results == 0)
print(fail_idx2[0])
print(len(fail_idx2[0]))

In [ ]:
for i in range(len(fail_idx2[0])):

    temp_img = cv2.imread(te_data_dir[fail_idx2[0][i]][0])
    
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.axis("off")
    plt.imshow(cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB))
    plt.show()

안 찢어진 사진 25개 중 18개를 찢어졌다고 판단
찢어진 사진 50개 중 1개를 안 찢어졌다고 판단